In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

sns.set()

In [ ]:
# define parameters for SCB's JSON API for population data #
# don't forget to change the default 'px' response format to 'json' bottom of the pop_query data struct # 

pop_url = 'http://api.scb.se/OV0104/v1/doris/en/ssd/START/BE/BE0101/BE0101A/FolkmangdNov'

pop_query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
### function to fetch SCB population or death data, returns 3 DataFrames w. different age binnings ### 

def fetch_scb_data(url,query,param='pop'):
    
    # call the API, check HTML status code (should be 200 for success) #

    r= requests.post(url,json=query)
    print ('HTML status code fetching {}: '.format(param),r.status_code)
    
    # extract the json data section from the response #

    json_data = r.json()['data'] # get the data section (skip metadata) #

    # we get two dictionaries : 'key' and 'values' #
    # 'key' attributes are : 'geo' (all of sweden), 'age','gender','year' #
    # 'values' attribute is : 'population' / 'dead' #
    
    # check the first record to figure out the structure of the data #
    print ('record layout for {}: '.format(param),json_data[0])
    
    # create a Pandas DataFrame from the list of dictionaries, and massage it to a decent structure #

    age_df = pd.DataFrame.from_dict(json_data)
    age_df[['area','age','gender','year']] = age_df['key'].to_list() # split keys to separate columns #

    age_df[param] = age_df['values'].apply(
        lambda x : x[0]).astype(int) # extract size from the values list #

    age_df['num_age'] = age_df['age'].apply(
        lambda x :x.replace('+','')).astype(int) # create numeric age column #

    age_df = age_df.drop(['key','values','area'],axis=1) # delete unwanted columns #

    # combine the two genders #
    age_df = age_df.groupby(['year','num_age']).sum()

    # bin the data to the same age bins that are used in SCB's prel. death data #
    age_df['scb_prel_age_bin'] = pd.cut(age_df.index.get_level_values(1),[-1,64,79,89,200],
                                        labels=['-64','65-79','80-89','90+'])
    
    # and finally, for each year, combine the 1 year age bin sums to match the 4 age bins of the prel death data #

    age_scb_prel_df = age_df.groupby([age_df.index.get_level_values(0),
                                                      'scb_prel_age_bin']).sum()
    
    # create a df with total yearly values #
    df = age_scb_prel_df.groupby('year').sum()
    
    
    

    return age_df,age_scb_prel_df,df
    

In [ ]:
bin_1_pop,bin_4_grp_pop,no_bin_pop = fetch_scb_data(pop_url,pop_query)



In [ ]:
bin_4_grp_pop = bin_4_grp_pop.unstack().droplevel(0,axis=1)
bin_1_pop = bin_1_pop.drop('scb_prel_age_bin',axis=1).unstack().droplevel(0,axis=1)


In [ ]:
bin_4_grp_pop

In [ ]:
'''
scb_prel_url = 'https://www.scb.se/hitta-statistik/statistik-efter-amne/befolkning/'\
'befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/'
'''
scb_prel_url = "https://scb.se/link/0fec627c501e476aacf7ae328c7a4a8b.aspx"

scb_prel_file = pd.ExcelFile(scb_prel_url)


In [ ]:
last_reported_date = scb_prel_file.parse('Info',usecols=[15]).dropna()
last_reported_date

In [ ]:
'''
scb_prel_url = 'https://www.scb.se/hitta-statistik/statistik-efter-amne/befolkning/'\
'befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/'
'''

scb_prel_file = pd.ExcelFile(scb_prel_url)

scb_prel_age = scb_prel_file.parse('Tabell 2',skiprows=7,usecols=[0,1,4,5,6,7,9,10,11,12])

scb_prel_age['DagMånad'] = scb_prel_age['DagMånad'].str.strip(' ')

unknowns = scb_prel_age.loc[scb_prel_age['DagMånad'] == 'Okänd dödsdag']

scb_prel_age = scb_prel_age.loc[scb_prel_age['DagMånad'] != 'Okänd dödsdag']
scb_prel_age = scb_prel_age[::-1]

scb_prel_age[['day','month']] = scb_prel_age['DagMånad'].str.split(expand=True)

swe_to_eng = {'januari':1,'februari' : 2,'mars' : 3,'april' : 4,
              'maj' : 5,'juni' : 6,'juli' : 7,'augusti' : 8,
              'september' : 9,'oktober' : 10,'november' : 11,'december' : 12}

scb_prel_age.rename(columns={'År' : 'year'},inplace=True)

scb_prel_age['day'] = scb_prel_age['day'].astype(int)
scb_prel_age['month'] = scb_prel_age['month'].apply(lambda x : int(swe_to_eng[x]))

scb_prel_age.index = pd.to_datetime(scb_prel_age[['year','month','day']])

scb_prel_age.drop(['year','DagMånad','day','month'],axis=1,inplace=True)

unknowns = unknowns.set_index('År')

unknowns.drop('DagMånad',axis=1,inplace=True)

### pattern transform ###
day_proportion = scb_prel_age.groupby(scb_prel_age.index.year).transform(lambda x : x / x.sum())

### pattern different size df's ### 
# in order to multiply day_proportions with unknowns, year-by-year, we must temporarily set a common index.
# the index will contain duplicates of years, that's no problem # 

idx = day_proportion.index
day_proportion = day_proportion.set_index(day_proportion.index.year)

add_ons = day_proportion * unknowns
add_ons.index = idx

scb_prel_age += add_ons

scb_prel_age

In [ ]:
# sanity check #
scb_prel_age.groupby(scb_prel_age.index.year).sum().sum(axis=1)

In [ ]:
def combine_genders(col):
    if '64' in col:
        return '-64'
    if '79' in col:
        return '65-79'
    if '89' in col:
        return '80-89'
    if '90' in col:
        return '90+'
    
combined_gender_deaths = scb_prel_age.groupby(combine_genders,axis=1).sum()



In [ ]:
combined_gender_deaths

In [ ]:
# sanity check #
combined_gender_deaths.groupby(combined_gender_deaths.index.year).sum().sum(axis=1)

In [ ]:
### USING PREVIOUS YEAR'S POP AS POP ###
bin_4_grp_pop = bin_4_grp_pop.loc['2014':]
bin_4_grp_pop.index = range(2015,2022)
bin_4_grp_pop

In [ ]:
full_year_deaths = combined_gender_deaths.resample('Y').sum()
full_year_deaths.index = full_year_deaths.index.year
full_year_deaths

In [ ]:
full_year_mortality = full_year_deaths / bin_4_grp_pop
full_year_mortality

In [ ]:
baseline_start = '2015'
baseline_end = '2018'

In [ ]:
full_year_mortality_baseline = full_year_mortality.loc[baseline_start : baseline_end].mean()
full_year_mortality_baseline

In [ ]:
full_year_expectation = full_year_mortality_baseline * bin_4_grp_pop
full_year_expectation

In [ ]:
##### full year summary ######

full_year_excess = full_year_deaths - full_year_expectation
full_year_excess_totals = full_year_excess.sum(axis=1)
full_year_excess_totals

In [ ]:
#### PATTERN to combine df's with different sizes - temporarily create common index ####
daily_mortality = (combined_gender_deaths.set_index(combined_gender_deaths.index.year) / bin_4_grp_pop).set_index(
    combined_gender_deaths.index)

daily_mortality

In [ ]:
daily_baseline_data = daily_mortality.loc[baseline_start : baseline_end]
daily_baseline_mortality = daily_baseline_data.groupby(
    [daily_baseline_data.index.month,daily_baseline_data.index.day]).mean()
daily_baseline_mortality

In [ ]:
leap_year_daily_mortality = daily_baseline_mortality.copy()
leap_year_daily_mortality.index = np.arange(1,367)
non_leap_year_daily_mortality = daily_baseline_mortality.copy().drop((2,29))
non_leap_year_daily_mortality.index = np.arange(1,366)

print (non_leap_year_daily_mortality.tail())
print ()
print (leap_year_daily_mortality.tail())

print (leap_year_daily_mortality.info())

In [ ]:
ax = non_leap_year_daily_mortality[50:70].plot(figsize=(18,12),subplots=True,style='x--')
leap_year_daily_mortality[50:70].plot(subplots=True,ax=ax,style='o--')

In [ ]:
timeline_daily_baseline = pd.concat([non_leap_year_daily_mortality,leap_year_daily_mortality,
                                     non_leap_year_daily_mortality,non_leap_year_daily_mortality,
                                     non_leap_year_daily_mortality,leap_year_daily_mortality,
                                     non_leap_year_daily_mortality],axis=0)

timeline_daily_baseline.index = pd.date_range(baseline_start,periods=len(timeline_daily_baseline))
timeline_daily_baseline


In [ ]:
axes = daily_mortality.plot(figsize=(18,12),subplots=True)
timeline_daily_baseline.plot(subplots=True,color='lightgrey',ls='--',figsize=(18,12),ax=axes)

In [ ]:
bin_4_grp_pop

In [ ]:
##### PATTERN #####
daily_expected_deaths = (timeline_daily_baseline.set_index(
    timeline_daily_baseline.index.year) * bin_4_grp_pop).set_index(
    timeline_daily_baseline.index)

daily_expected_deaths

In [ ]:
full_year_expectation_by_daily = daily_expected_deaths.groupby(daily_expected_deaths.index.year).sum()
full_year_expectation_by_daily

In [ ]:
full_year_expectation_by_daily.sum(axis=1)

In [ ]:
full_year_expectation.sum(axis=1)

In [ ]:
### leap years stand out ! ####
full_year_expectation.sum(axis=1) - full_year_expectation_by_daily.sum(axis=1)

In [ ]:
excess_by_full_year = combined_gender_deaths.groupby(combined_gender_deaths.index.year).sum() - full_year_expectation
excess_by_full_year.sum(axis=1)

In [ ]:
excess_by_daily = combined_gender_deaths.groupby(combined_gender_deaths.index.year).sum() - full_year_expectation_by_daily
excess_by_daily.sum(axis=1)

In [ ]:
ax = excess_by_full_year[:-1].sum(axis=1).plot(style='o--')
excess_by_daily[:-1].sum(axis=1).plot(ax=ax,style='o--')

In [ ]:
(full_year_expectation.sum(axis=1) / full_year_expectation_by_daily.sum(axis=1)).plot(kind='bar')
plt.ylim([0.99,1.001])

In [ ]:
no_leap_baseline = daily_mortality.loc['2017' : '2018'].groupby(daily_mortality.loc['2017':'2018'].index.dayofyear).mean()
no_leap_baseline

In [ ]:
no_leap_timeline = pd.concat([no_leap_baseline] * 3)
no_leap_timeline.index = pd.date_range('2017-01-01','2019-12-31')
no_leap_timeline

In [ ]:
no_leap_daily_expectation = (no_leap_timeline.set_index(no_leap_timeline.index.year) * bin_4_grp_pop).dropna().set_index(no_leap_timeline.index)
no_leap_daily_expectation

In [ ]:
no_leap_yearly_expectation_by_days = no_leap_daily_expectation.groupby(no_leap_daily_expectation.index.year).sum()
no_leap_yearly_expectation_by_days.sum(axis=1)

In [ ]:
no_leap_data_baseline = combined_gender_deaths.loc['2017' : '2018']
no_leap_full_year_base = (no_leap_data_baseline.groupby(no_leap_data_baseline.index.year).sum() / bin_4_grp_pop).dropna().mean()

In [ ]:
no_leap_full_year_base

In [ ]:
no_leap_full_year_expectation = no_leap_full_year_base * bin_4_grp_pop
no_leap_full_year_expectation

In [ ]:
no_leap_full_year_expectation.sum(axis=1)

In [ ]:
no_leap_yearly_expectation_by_days.sum(axis=1)

In [ ]:
#################################################################################
leap_year_subset = combined_gender_deaths.loc[combined_gender_deaths.index.is_leap_year]
leap_days = leap_year_subset.loc[(leap_year_subset.index.month==2) & (leap_year_subset.index.day==29)]
leap_days

In [ ]:
combined_gender_deaths_no_leap_days = combined_gender_deaths.copy().drop(leap_days.index)

In [ ]:
combined_gender_deaths

In [ ]:
combined_gender_deaths_no_leap_days.loc['2016']

In [ ]:
daily_death_proportions = combined_gender_deaths_no_leap_days.groupby(
    combined_gender_deaths_no_leap_days.index.year).transform(lambda x : x / x.sum())

daily_death_proportions

In [ ]:
add_ons_2016 = daily_death_proportions.loc['2016']  * leap_days.loc['2016-02-29']
add_ons_2020 = daily_death_proportions.loc['2020'] * leap_days.loc['2020-02-29']

add_ons_2020

In [ ]:
combined_gender_deaths_no_leap_days.loc['2016'] += add_ons_2016

combined_gender_deaths_no_leap_days.loc['2016']

In [ ]:
combined_gender_deaths_no_leap_days.loc['2020'] += add_ons_2020
combined_gender_deaths_no_leap_days.loc['2020']

In [ ]:
combined_gender_deaths_no_leap_days.groupby(combined_gender_deaths_no_leap_days.index.year).sum().sum(axis=1)

In [ ]:
combined_gender_deaths.groupby(combined_gender_deaths.index.year).sum().sum(axis=1)

In [ ]:
timeline_no_leap = pd.concat([no_leap_baseline] * 7)

In [ ]:
365 * 7

In [ ]:
idx_no_leap_days = pd.date_range('2015-01-01','2021-12-31')
leap_mask = (idx_no_leap_days == '2016-02-29') | (idx_no_leap_days == '2020-02-29')
idx_no_leap_days = idx_no_leap_days[~leap_mask]
idx_no_leap_days

In [ ]:
### this timeline has the 2 leap day deaths distributed proportionally over their resp. years ###
timeline_no_leap.index = idx_no_leap_days
timeline_no_leap

In [ ]:
Expected_daily = (timeline_no_leap.set_index(timeline_no_leap.index.year) * bin_4_grp_pop).set_index(
    timeline_no_leap.index)

Expected_daily

In [ ]:
Expected_daily_no_leap = Expected_daily.groupby(Expected_daily.index.year).sum().sum(axis=1)
Expected_daily_no_leap

In [ ]:
full_year_mort_no_leap = combined_gender_deaths_no_leap_days.groupby(combined_gender_deaths_no_leap_days.index.year).sum() / bin_4_grp_pop

full_baseline_no_leap = full_year_mort_no_leap.loc['2015' : '2018'].mean()
full_baseline_no_leap

In [ ]:
Expected_full_no_leap = (full_baseline_no_leap * bin_4_grp_pop).sum(axis=1)
Expected_full_no_leap

In [ ]:
Expected_daily_no_leap/ Expected_full_no_leap

In [ ]:
yearly_actuals = combined_gender_deaths_no_leap_days.groupby(combined_gender_deaths_no_leap_days.index.year).sum()
Excess_daily_no_leap = yearly_actuals.sum(axis=1) - Expected_daily_no_leap
Excess_daily_no_leap

In [ ]:
Excess_full_no_leap = yearly_actuals.sum(axis=1) - Expected_full_no_leap
Excess_full_no_leap

In [ ]:
################################################################################

df_no_leap = pd.DataFrame({'dead' : np.repeat(1,365),
                          'pop' : np.repeat(1000,365)})


df_no_leap['mortality'] = df_no_leap['dead'] / df_no_leap['pop']
df_no_leap

In [ ]:
print (df_no_leap.mean())
print()
print (df_no_leap[['dead','mortality']].sum())

In [ ]:
df_leap = pd.DataFrame({'dead' : np.repeat(1,366),
                       'pop' : np.repeat(1000,366)})

df_leap['mortality'] = df_leap['dead'] / df_leap['pop']

print (df_leap.mean())
print ()
print (df_leap[['dead','mortality']].sum())

In [ ]:
Expected_daily

In [ ]:
combined_gender_deaths_no_leap_days

In [ ]:
daily_excess = combined_gender_deaths_no_leap_days - Expected_daily
daily_excess

In [ ]:
### find out the last reported day of 2021, and define last_good_day ###

last_data_day = daily_excess.dropna().index[-1]
last_good_day = last_data_day - pd.DateOffset(weeks=2)
last_good_day

In [ ]:
title = 'SWEDEN yearly cumulative excess deaths [baseline: daily mortality 2015-2018]\nYTD {}\nDataSource : scb.se'.format(
last_data_day.date())

cum_daily_excess_per_year = daily_excess.groupby(daily_excess.index.year).transform(lambda x : x.cumsum())

axes = cum_daily_excess_per_year.plot(subplots=True,figsize=(18,12),sharey=True,title=title)

for a in axes:
    a.axvline(last_good_day,ls='dashed',color='orange',label='last stable data {}'.format(last_good_day.date()))
    a.axhline(0,ls='dashed',color='k')
    a.set_ylabel('excess deaths')
    a.legend(loc='lower left')

plt.savefig('excess_deaths_full_vs_daily_cum_excess_deaths.jpg',format='jpg')

In [ ]:
daily_excess.groupby(daily_excess.index.year).sum()

In [ ]:
#total excess YTD calculated by daily mortality (vs excess 2020 of 2019 if calculated by yearly mortality ) 
daily_excess.groupby(daily_excess.index.year).sum().sum(axis=1)

In [ ]:
title = 'SWEDEN daily mortality vs baseline [2015-2018]\nYTD {}\nDataSource : scb.se'.format(
last_data_day.date())

axes = daily_mortality.plot(figsize=(18,12),subplots=True,title=title)
timeline_daily_baseline.plot(subplots=True,color='lightgrey',ls='--',figsize=(18,12),ax=axes,label='baseline')

for a in axes:
    a.axvline(last_good_day,ls='dashed',color='orange',label='last stable data {}'.format(last_good_day.date()))
    a.set_ylabel('daily mortality')
    a.legend(loc='lower left')
    
plt.savefig('excess_deaths_full_vs_daily_daily_mort.jpg',format='jpg')